In [3]:
%pip install plotly pandas
%pip install --upgrade nbformat
%pip install nbformat
%pip install plotly
%pip install -U kaleido

import warnings
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
warnings.filterwarnings("ignore", message=".*scattermapbox.*", category=DeprecationWarning)


df = pd.read_csv("Chosen_Tracts.csv")
df_clean = df[['longitude', 'latitude', 'poverty population', 'CensusTract']].dropna()

coords = df_clean[['longitude', 'latitude']].to_numpy()
weights = df_clean['poverty population'].to_numpy()

k = int(input("Enter the number of facilities to place: "))
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(coords, sample_weight=weights)

df_clean['cluster'] = kmeans.predict(coords)
df['cluster'] = df_clean['cluster']

closest_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, coords)
selected_tracts = df_clean.iloc[closest_indices]['CensusTract'].values
centers = kmeans.cluster_centers_

print("\nFacility locations (closest CensusTract IDs):")
for i, tract in enumerate(selected_tracts, start=1):
    print(f"Facility {i}: CensusTract {tract}")

facilities = pd.DataFrame({
    'CensusTract': selected_tracts,
    'longitude': centers[:, 0],
    'latitude': centers[:, 1]
})
gdf_facilities = gpd.GeoDataFrame(
    facilities,
    geometry=gpd.points_from_xy(facilities['longitude'], facilities['latitude']),
    crs="EPSG:4326"
).to_crs(epsg=3857)


fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "mapbox", "colspan": 2}, None],
           [{"type": "mapbox"}, {"type": "mapbox"}]],
    row_heights=[0.5, 0.5],
    vertical_spacing=0.005,
    column_widths=[0.5, 0.5],
    subplot_titles=["Continental US", "Alaska", "Hawaii"]
)

for i, row in facilities.iterrows():
    fig.add_trace(
        go.Scattermapbox(
            lat=[row['latitude']],
            lon=[row['longitude']],
            mode='markers',
            marker=dict(size=10, color='#58B8E4'),
            name=f"Facility {i+1}",
            hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=1, col=1  # All facilities shown on US map
    )
    fig.add_trace(
        go.Scattermapbox(
            lat=[row['latitude']],
            lon=[row['longitude']],
            mode='markers',
            marker=dict(size=10, color='#58B8E4'),    
            name=f"Facility {i+1}",
            hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=2, col=1
    )
        
    fig.add_trace(
        go.Scattermapbox(
        lat=[row['latitude']],
        lon=[row['longitude']],
        mode='markers',
        marker=dict(size=10, color='#58B8E4'),
        name=f"Facility {i+1}",
        hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=2, col=2
    )

fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center=dict(lat=37.5, lon=-95),
        zoom=2.5
    ),
    mapbox2=dict(
        style="carto-positron",
        center=dict(lat=64, lon=-150),
        zoom=1.5
    ),
    mapbox3=dict(
        style="carto-positron",
        center=dict(lat=20.5, lon=-157.5),
        zoom=4
    ),
    mapbox_domain=dict(row=1, column=1),
    mapbox2_domain=dict(row=2, column=1),
    mapbox3_domain=dict(row=2, column=2),
    height=800,
    title_text="Facility Placement Map: US, Alaska, and Hawaii",
    showlegend=False
)

fig.show()
facilities.to_csv('optimal_facility_locations.csv', index=False)
print(f"\n✅ Saved {k} facility locations to 'optimal_facility_locations.csv'")

/var/folders/rn/gtj4kyrs66s7lwh782tpf_bw0000gn/T/ipykernel_8907/3392846226.py:4: DtypeWarning:

Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.




Facility locations (closest CensusTract IDs):
Facility 1: CensusTract 17141960700.0
Facility 2: CensusTract 6029001000.0
Facility 3: CensusTract 12105014202.0
Facility 4: CensusTract 48051970100.0
Facility 5: CensusTract 34031256805.0
Facility 6: CensusTract 35045942900.0
Facility 7: CensusTract 1127021300.0
Facility 8: CensusTract 53039950200.0
Facility 9: CensusTract 37057062001.0
Facility 10: CensusTract 20139010500.0
Facility 11: CensusTract 15009031801.0
Facility 12: CensusTract 39175938400.0
Facility 13: CensusTract 2170000502.0



✅ Saved 13 facility locations to 'optimal_facility_locations.csv'
